# ChatGPTでfunction callingを用いて過去の会話履歴を呼び出す

もしマイ・チャットボットがいたら、当然、数日前の会話内容を覚えていて欲しいですよね？  
以下では、過去の会話内容に基づいた会話をするチャットボットを、OpenAI APIのfunction callingを用いて作っていきます。

1. 環境構築
2. 会話内容の保存・読込
3. function callingを用いて過去の会話情報を呼び出す

## 1. 環境構築

In [1]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00


In [8]:
import os
os.environ['OPENAI_API_KEY'] = 'PLEASE CHANGE HERE'

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

## 2. 会話内容の保存・読込

### 会話内容の保存・読込

In [3]:
import json
from datetime import datetime

def save_conversation_from_messages(outdir, datetime_str, messages, **kwargs):
    chat_datetime = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    json_filepath = os.path.join(outdir, 'chat_history_{}.json'.format(chat_datetime.strftime('%Y%m%d')))
    dat = dict({'datetime': datetime_str, 'messages': messages}, **kwargs)  # 任意の情報を保存可能
    with open(json_filepath, 'w') as fout:
        json.dump(dat, fout)
    print('Saved ...', json_filepath)

def load_conversation_from_json(json_filepath):
    with open(json_filepath, 'r') as fin:
        dict_all = json.load(fin)
    return dict_all

### 実験用に、会話内容を保存する

In [4]:
# json_file, datetime, chat_history
past_conversations = [
    [
        '2023-06-28 19:03:42',
        [
            {
                "role": "user",
                "content": "最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。",
            },
            {
                "role": "assistant",
                "content": "いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？",
            },
        ]
    ],
    [
        '2023-07-03 21:53:02',
        [
            {
                "role": "user",
                "content": "今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。",
            },
            {
                "role": "assistant",
                "content": "いいなぁ。僕もイタリアン食べたい！！",
            },
        ]
    ],
    [
        '2023-07-04 20:11:56',
        [
            {
                "role": "user",
                "content": "今日の昼は中華レストランでよだれ鶏を食べんたんだ。最近暑いからか、あの酸味と辛味を欲してしまうんだよね",
            },
            {
                "role": "assistant",
                "content": "暑いと酸味と辛味を欲するの？なんで？",
            },
        ]
    ],
]

# 日別で会話内容を保存
for datetime_str, messages in past_conversations:
    save_conversation_from_messages('./', datetime_str, messages)

Saved ... ./chat_history_20230628.json
Saved ... ./chat_history_20230703.json
Saved ... ./chat_history_20230704.json


In [5]:
# 試しに読み込む
load_conversation_from_json('chat_history_20230628.json')

{'datetime': '2023-06-28 19:03:42',
 'messages': [{'role': 'user',
   'content': '最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。'},
  {'role': 'assistant', 'content': 'いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？'}]}

## 3. function callingを用いた方式
1. function callingを使用してみる
2. 返値を`"role": "function"`として渡してみる

In [10]:
class MyChatBot():

    SYSTEM_PROMPT = """You are a helpfule assistant.
You need to follow the following rules:
- lang:ja
- please be polite (e.g. use です, ます)
- short reply (less than 100 japanese charactors)
"""

    # NOTE デフォルトでGPT-4を使用する
    def __init__(self, model='gpt-4', temparature=0.0):
        self._model = model
        self._temparature = temparature
        self.reset_messages()

        # For function calling
        self._available_functions = {}
        self._function_properties = {}

    def reset_messages(self):
        self._messages = [{"role": "system", "content": MyChatBot.SYSTEM_PROMPT}]

    def get_messages(self):
        return self._messages.copy()

    def add_function(self, function, func_props):
        if not callable(function):
            raise TypeError('function must be callable.')
        func_name = func_props['name']
        self._available_functions[func_name] = function
        self._function_properties[func_name] = func_props

    @property
    def function_properties(self):
        if len(self._function_properties) == 0:
            return None
        return list(self._function_properties.values())

    def chat(self, input, verbose=False):
        user_message = {'role': 'user', 'content': input}
        response = self._chat_completion(user_message)

        # add assistant message
        response_message = response['choices'][0]['message']
        self._messages.append(response_message)

        # Function Calling
        if response_message.get('function_call'):
            function_name = response_message['function_call']['name']

            fuction_to_call = self._available_functions[function_name]
            function_args = json.loads(response_message['function_call']['arguments'])
            function_response = fuction_to_call(**function_args)

            if verbose:
                print(f'request function: {function_name}')
                print('  args - ', function_args)

            function_message = {
                'role': 'function',
                "name": function_name,
                "content": function_response,
            }
            response = self._chat_completion(function_message)

        return response

    def _chat_completion(self, message, enable_functions=True):
        # update messages
        self._messages.append(message)

        # for function calling
        additional_args = {}
        if enable_functions and (len(self._function_properties) > 0):
            additional_args['functions'] = list(self._function_properties.values())

        # run chat-completion
        response = openai.ChatCompletion.create(
            model=self._model,
            temperature=self._temparature,
            messages=self._messages,
            **additional_args
        )
        return response

In [12]:
my_chatbot = MyChatBot()
response = my_chatbot.chat('こんにちは')

print(response)
print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-7zyMj76OToOV6rd1oiHftwNCdOk2Q",
  "object": "chat.completion",
  "created": 1695004409,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u3053\u3093\u306b\u3061\u306f\uff01\u4f55\u304b\u304a\u624b\u4f1d\u3044\u3067\u304d\u308b\u3053\u3068\u304c\u3042\u308a\u307e\u3059\u304b\uff1f"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 60,
    "completion_tokens": 19,
    "total_tokens": 79
  }
}
こんにちは！何かお手伝いできることがありますか？


In [13]:
import re

def recall_past_chats(the_date):
    if re.match(r'\d{4}-\d{2}-\d{2}', the_date) is None:  # is not date format
        return None
    json_file = 'chat_history_{}.json'.format(the_date.replace('-', ''))
    chats_history = load_conversation_from_json(json_file)
    messages = chats_history['messages']
    # make output
    output = f"The following is chats on {the_date}　between the user and YOU (i.e. assistant);"
    for msg in messages:
        role = msg['role']
        content = msg['content']
        output += f"\n{role}: {content}"
    return output

func_props = {
    'name': 'recall_past_chats',
    'description': 'Access the past chats between you and the user. Today is 2023-07-05.',
    "parameters": {
        "type": "object",
        "properties": {
            "the_date": {
                "type": "string",
                "description": "Specify the date of the chats you need to access in the format 'YYYY-MM-DD'",
            },
        },
        "required": ["the_date"],
    }
}

# function callingを有効にして会話する
my_chatbot = MyChatBot()
my_chatbot.add_function(recall_past_chats, func_props)

response = my_chatbot.chat('一昨日僕が食べた料理を答えてみて。')

print(response)
print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-7zyMujwsbS1Z1A8IPokcyb6ASNzqU",
  "object": "chat.completion",
  "created": 1695004420,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u4e00\u6628\u65e5\u3001\u3042\u306a\u305f\u306f\u53cb\u9054\u3068\u30a4\u30bf\u30ea\u30a2\u30f3\u306b\u884c\u304d\u3001\u30a4\u30ab\u30b9\u30df\u30d1\u30b9\u30bf\u3068\u771f\u9bdb\u306e\u30ab\u30eb\u30d1\u30c3\u30c1\u30e7\u3092\u98df\u3079\u307e\u3057\u305f\u306d\u3002\u3069\u3061\u3089\u3082\u7f8e\u5473\u3057\u304b\u3063\u305f\u3068\u306e\u3053\u3068\u3067\u3059\u3002"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 298,
    "completion_tokens": 68,
    "total_tokens": 366
  }
}
一昨日、あなたは友達とイタリアンに行き、イカスミパスタと真鯛のカルパッチョを食べましたね。どちらも美味しかったとのことです。


### 実験

In [14]:
def chat_experiment(input):
    my_chatbot = MyChatBot()
    my_chatbot.add_function(recall_past_chats, func_props)
    response = my_chatbot.chat(input, verbose=True)
    response_content = response['choices'][0]['message']['content']
    return response_content

In [15]:
# GPT-3.5 : 一昨日、あなたはイカスミパスタと真鯛のカルパッチョを食べました。
# GPT-4   : 一昨日、あなたは友達とイタリアンに行き、イカスミパスタと真鯛のカルパッチョを食べましたね。どちらも美味しかったとのことです。
chat_experiment(input='一昨日僕が食べた料理を答えてみて。')

request function: recall_past_chats
  args -  {'the_date': '2023-07-03'}


'一昨日、あなたは友達とイタリアンに行き、イカスミパスタと真鯛のカルパッチョを食べましたね。どちらも美味しかったとのことです。'

In [16]:
# GPT-3.5 : 7/3には、イカスミパスタと真鯛のカルパッチョを食べたそうです。どちらも美味しかったとのことですね！
# GPT-4   : 7月3日には、イカスミパスタと真鯛のカルパッチョを食べたとのことですね。どちらも美味しかったとのこと、良かったですね。
chat_experiment(input="7/3に僕が食べた料理を答えてみて。")

request function: recall_past_chats
  args -  {'the_date': '2023-07-03'}


'7月3日には、イカスミパスタと真鯛のカルパッチョを食べたとのことですね。どちらも美味しかったとのこと、良かったですね。'

In [17]:
# GPT-3.5 : それは素晴らしいですね！昨日食べた料理を自分でも作ってみたのですね。では、昨日食べた料理は何でしたか？
# GPT-4   : よだれ鶏を作ったのですね。お疲れ様でした。美味しくできましたか？
chat_experiment(input="昨日食べた料理が美味しかったから、自分でも作ってみたよ。何を作ったでしょうか?")

request function: recall_past_chats
  args -  {'the_date': '2023-07-04'}


'よだれ鶏を作ったのですね。お疲れ様でした。美味しくできましたか？'

In [18]:
# GPT-3.5 : 昨日の料理は「よだれ鶏」でしたね。
# GPT-4   : 昨日の昼食は中華レストランでよだれ鶏を召し上がったようですね。
chat_experiment(input="昨日食べた料理は何でしょうか?")

request function: recall_past_chats
  args -  {'the_date': '2023-07-04'}


'昨日の昼食は中華レストランでよだれ鶏を召し上がったようですね。'

In [19]:
# GPT-3.5 : 7月4日に食べた料理は「よだれ鶏」でした。暑い日には、酸味と辛味を欲することがあるんですよね。
# GPT-4   : 7月4日には中華レストランでよだれ鶏を召し上がったようですね。
chat_experiment(input="7/4に食べた料理は何でしょうか？")

request function: recall_past_chats
  args -  {'the_date': '2023-07-04'}


'7月4日には中華レストランでよだれ鶏を召し上がったようですね。'

In [20]:
# GPT-3.5 : 一週間前の2023年6月28日には、冷麺を食べたようですね。具体的には、盛岡冷麺か韓国冷麺のどちらかを食べたのか、その情報はありませんでした。
# GPT-4   : 一週間前の2023年6月28日に、冷麺を召し上がったようですね。
chat_experiment(input="僕、一週間前って何食べたっけ？")

request function: recall_past_chats
  args -  {'the_date': '2023-06-28'}


'一週間前の2023年6月28日に、冷麺を召し上がったようですね。'

### 実験（リベンジ）
- GPT-3.5で回答に失敗した質問を、少し表現を変えてリベンジ
- 結果
    - GPT-3.5 : 変わらず失敗
    - GPT-4 : 元々成功。表現を変更しても問題なく成功

In [23]:
# GPT-3.5 : それは素晴らしいですね！では、昨日作った料理は何でしょうか？楽しみにしています！
# GPT-4   : よだれ鶏を作ったのではないでしょうか。
chat_experiment(input="昨日食べた料理が美味しかったから、自分でも作ってみたよ。何を作ったか当ててみて！")

request function: recall_past_chats
  args -  {'the_date': '2023-07-04'}


'よだれ鶏を作ったのではないでしょうか。'

In [24]:
# GPT-3.5 : 一昨日食べた料理が美味しかったのですね。それでは、一昨日作った料理を再現してみたのですか？どの料理を作ったのか教えていただけますか？
# GPT-4   : イカスミパスタと真鯛のカルパッチョを作ったのですね。どちらも美味しそうですね！
chat_experiment(input="一昨日食べた料理が美味しかったから、自分でも作ってみたよ。何を作ったでしょうか?")

request function: recall_past_chats
  args -  {'the_date': '2023-07-03'}


'イカスミパスタと真鯛のカルパッチョを作ったのですね。どちらも美味しそうですね！'

## おまけ

In [21]:
# function calling の description に今日の日付の情報が入っているので、日付を回答できている？
# GPT-3.5 : 一昨日は2023年7月3日です。
# GPT-4   : 一昨日は2023年7月3日でした。
chat_experiment(input="一昨日は何月何日？")

'一昨日は2023年7月3日でした。'

In [22]:
# GPT-3.5 : 今日は2023年7月です。
# GPT-4   : 今日は7月です。
chat_experiment(input="今日は何月ですか？")

'今日は7月です。'